<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/1-introduction-sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción

Descargamos los datos necesarios para las actividades

In [1]:
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/rae.txt
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/pacientesA.txt
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/pacientesB.txt
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/proveedoresA.txt
!wget https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/access.log

--2020-11-03 02:26:01--  https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/rae.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145 [text/plain]
Saving to: ‘rae.txt’

rae.txt             100%[===================>]     145  --.-KB/s    in 0s      

2020-11-03 02:26:02 (6.76 MB/s) - ‘rae.txt’ saved [145/145]

--2020-11-03 02:26:02--  https://raw.githubusercontent.com/fvillena/dcc-ia-nlp/master/data/pacientesA.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33 [text/plain]
Saving to: ‘pacientesA.txt’

pacientesA.txt     

## Actividad 1: RAE

Desde el archivo rae.txt almacene la palabra y sus significados en una lista y en un diccionario. Implemente una función que dada una palabra retorne los significados. Compare los tiempos de respuesta al realizar la búsqueda sobre la lista y el diccionario.

In [2]:
# Así leemos archivos de texto
rae_list = []
rae_dict = {}
with open("rae.txt", "r") as f:
  for raw_line in f:
    line = raw_line.rstrip() # el método String.rstrip() elimina caracteres innecesarios al final del string.
    entry = line.split("\t")
    rae_list.append(entry)
    rae_dict[entry[0]] = entry[1:]

In [3]:
# Prográmame
def definitions_from_rae_list(word):
  for entry in rae_list:
    if entry[0] == word:
      return entry[1:]
      break
  raise Exception("word not found")
def definitions_from_rae_dict(word):
  try:
    return rae_dict[word]
  except KeyError:
    raise Exception("word not found")

In [4]:
print(definitions_from_rae_list("lapiz"))
print(definitions_from_rae_dict("sacapunta"))

['elemento para escribir', 'elemento para dibujar']
['elemento para sacarpunta', 'arma de ataque']


Programe una función que dado un archivo de texto, una palabra y un par de definiciones, el archivo de texto sea actualizado agregando la palabra y definiciones nuevas.

In [5]:
# Prográmame
def update_rae(rae_file,word,definitions):
  entry = [word] + definitions
  line = "\t".join(entry)
  with open(rae_file, "a") as f:
    f.write(line)

In [6]:
update_rae("rae.txt","estuche",["envoltura para guardar ordenadamente un objeto o varios","envoltura que reviste y protege algo"])

## Actividad 2: Combinar archivos

Suponga que se tienen dos lista de pacientesA.txt y pacientesB.txt en un formato csv (nombre,rut).
Implemente un programa que cree un archivos con todos los pacientes, imprima el total de pacientes en A, B y la suma total.

In [7]:
# Prográmame
def combine_patient_lists(patient_lists,combined_list):
  data = {}
  for patient_list_file in patient_lists:
    patient_list = []
    with open(patient_list_file) as f:
      for line in f:
        patient = line.rstrip().split(",")
        patient_list.append(patient)
    data[patient_list_file] = patient_list
  patients_total = 0
  with open(combined_list,"w") as f:
    for file_name,patients in data.items():
      print(f"Pacientes en {file_name}: {str(len(patients))}")
      for patient in patients:
        f.write(",".join(patient) + "\n")
        patients_total += 1
  print(f"Total de pacientes: {patients_total}")

In [8]:
combine_patient_lists(["pacientesA.txt","pacientesB.txt"],"pacientesAB.txt")

Pacientes en pacientesA.txt: 3
Pacientes en pacientesB.txt: 4
Total de pacientes: 7


## Actividad 3: Fármacos

Desde el archivo proveedoresA.txt cree una función que lea cada linea y extraiga la información contenida en 5 grupos.

El formato de cada linea es el siguiente:

`PFZ-000000000001-001,Aspirinas 50 mg en paquetes de 25,800`

In [15]:
# Prográmame
import re
def parse_medication_list(medication_list):
  pattern = r"([A-Z]{3})-(\d{12})-(\d{3}),(.*),(\d+)"
  with open(medication_list) as f:
    for line in f:
      line = line.rstrip()
      match = re.search(pattern, line)
      medication = [group for group in match.groups()]
      print(medication)

In [16]:
parse_medication_list("proveedoresA.txt")

['PFZ', '000000000001', '001', 'Aspirinas 50 mg en paquetes de 25', '800']
['PFZ', '000000000001', '001', 'Aspirinas 50 mg en paquetes de 25', '2000']


## Actividad 4: Registro de acceso

Para el archivo adjunto access.log extraiga los IP desde donde se generan las visitas usando expresiones regulares en python. Con estas IP genere un archivo .txt, donde cada IP esta en una linea aparte. 
Con la lista de IP generada visualice en un mapa desde donde vienes las peticiones usando el sitio web https://www.showmyip.com/bulk-ip-lookup/, ¿cuántas visitas se originan desde rusia/ucrania/china/brasil?

HINT: Para diseñar su expresión regular puede utilizar https://regexr.com/ (u otro sitio de su preferencia).

In [23]:
# Prográmame
def ip_addresses_from_access_log(access_log,ip_addresses_file):
  with open(access_log) as f:
    pattern = r".* (\d+\.\d+\.\d+\.\d+).*"
    with open(ip_addresses_file,"w") as g:
      for line in f:
        line = line.rstrip()
        match = re.search(pattern, line)
        ip = match.group(1)
        g.write(f"{ip}\n")

In [24]:
ip_addresses_from_access_log("access.log","ip_addresses.txt")